In [1]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

from brian2 import *
from params import paramsJercog as p

import numpy as np

from network import JercogEphysNetwork
from results import ResultsEphys

%matplotlib
import matplotlib.pyplot as plt

INFO       Cache size for target "cython": 10481 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [3]:
p['setUpFRExc'] = 5 * Hz
p['setUpFRInh'] = 14 * Hz

p['threshExc'] = 130 * pA
p['threshInh'] = 140 * pA

p['gainExc'] = 0.49 * Hz / pA
p['gainInh'] = 1.46 * Hz / pA

In [4]:
p['recordStateVariables'] = ['v', 'sE', 'sI']
p['duration'] = 1 * second
p['nUnits'] = 1000
p['propConnect'] = 0.5

nExc = 400
nInh = 100
simType = 'poisson'  # poisson or fixed

In [5]:
defaultclock.dt = p['dt']
p['nInh'] = int(p['propInh'] * p['nUnits'])
p['nExc'] = int(p['nUnits'] - p['nInh'])
p['nExcSpikemon'] = int(p['nExc'] * p['propSpikemon'])
p['nInhSpikemon'] = int(p['nInh'] * p['propSpikemon'])
p['nIncInh'] = int(p['propConnect'] * p['propInh'] * p['nUnits'])
p['nIncExc'] = int(p['propConnect'] * (1 - p['propInh']) * p['nUnits'])

N = Network()

# NOTICE NO NOISE

unitModel = '''
                dv/dt = (gl * (eLeak - v) - iAdapt +
                         sE - sI) / Cm : volt (unless refractory)
                diAdapt/dt = -iAdapt / tauAdapt : amp

                dsE/dt = (-sE + uE) / tauFallE : amp
                duE/dt = -uE / tauRiseE : amp
                dsI/dt = (-sI + uI) / tauFallI : amp
                duI/dt = -uI / tauRiseI : amp
                
                eLeak : volt
                vReset : volt
                vThresh : volt
                betaAdapt : amp * second
                gl : siemens
                Cm : farad
                '''

resetCode = '''
        v = vReset
        iAdapt += betaAdapt / tauAdapt 
        '''

threshCode = 'v >= vThresh'

unitsExc = NeuronGroup(N=1, model=unitModel, method=p['updateMethod'], threshold=threshCode, reset=resetCode,
                       refractory=p['refractoryPeriodExc'], clock=defaultclock)
unitsInh = NeuronGroup(N=1, model=unitModel, method=p['updateMethod'], threshold=threshCode, reset=resetCode,
                       refractory=p['refractoryPeriodInh'], clock=defaultclock)

unitsExc.v = p['eLeakExc']
unitsExc.vReset = p['vResetExc']
unitsExc.vThresh = p['vThreshExc']
unitsExc.betaAdapt = p['betaAdaptExc']
unitsExc.eLeak = p['eLeakExc']
unitsExc.Cm = p['membraneCapacitanceExc']
unitsExc.gl = p['gLeakExc']

unitsInh.v = p['eLeakInh']
unitsInh.vReset = p['vResetInh']
unitsInh.vThresh = p['vThreshInh']
unitsInh.betaAdapt = p['betaAdaptInh']
unitsInh.eLeak = p['eLeakInh']
unitsInh.Cm = p['membraneCapacitanceInh']
unitsInh.gl = p['gLeakInh']

N.add(unitsExc, unitsInh)

In [6]:
if simType == 'poisson':  # make the Poisson spikers
    
    poissonExc = PoissonGroup(N=nExc, rates=p['setUpFRExc'], clock=defaultclock)
    poissonInh = PoissonGroup(N=nInh, rates=p['setUpFRInh'], clock=defaultclock)
    
elif simType == 'fixed':  # instead, make fixed rate that tiles the time period
    
    numSpikesExc = int(np.round(p['setUpFRExc'] * p['duration']))
    numSpikesInh = int(np.round(p['setUpFRInh'] * p['duration']))

    fixedIndicesExc = np.zeros((numSpikesExc, ), )
    fixedIndicesInh = np.zeros((numSpikesInh, ), )

    spikeTimesExc = np.linspace(0, p['duration'] / second, numSpikesExc + 1) * second
    spikeTimesExc = spikeTimesExc[:-1]

    spikeTimesInh = np.linspace(0, p['duration'] / second, numSpikesInh + 1) * second
    spikeTimesInh = spikeTimesInh[:-1]

    poissonExc = SpikeGeneratorGroup(N=1, indices=fixedIndicesExc, times=spikeTimesExc, clock=defaultclock)
    poissonInh = SpikeGeneratorGroup(N=1, indices=fixedIndicesInh, times=spikeTimesInh, clock=defaultclock)

N.add(poissonExc, poissonInh)

In [7]:
# make the synapses

onPreStrings = ('uE_post += jEE / tauRiseEOverMS',
                'uE_post += jIE / tauRiseEOverMS',
                'uI_post += jEI / tauRiseIOverMS',
                'uI_post += jII / tauRiseIOverMS',)

tauRiseEOverMS = p['tauRiseExc'] / ms
tauRiseIOverMS = p['tauRiseInh'] / ms
vTauExcOverMS = p['membraneCapacitanceExc'] / p['gLeakExc'] / ms
vTauInhOverMS = p['membraneCapacitanceInh'] / p['gLeakInh'] / ms

synapsesEE = Synapses(model='jEE: amp',
                     source=poissonExc,
                     target=unitsExc,
                     on_pre=onPreStrings[0])
synapsesIE = Synapses(model='jIE: amp',
                     source=poissonExc,
                     target=unitsInh,
                     on_pre=onPreStrings[1])
synapsesEI = Synapses(model='jEI: amp',
                     source=poissonInh,
                     target=unitsExc,
                     on_pre=onPreStrings[2])
synapsesII = Synapses(model='jII: amp',
                     source=poissonInh,
                     target=unitsInh,
                     on_pre=onPreStrings[3])

synapsesEE.connect()
synapsesIE.connect()
synapsesEI.connect()
synapsesII.connect()

In [8]:
useBetterWeights = True

if useBetterWeights:
    jEEmult = 1.0249649370098968
    jIEmult = 0.9913780811668145
    jEImult = 0.9750494640455015
    jIImult = 1.0108023944074438
else:
    jEEmult = 1
    jIEmult = 1
    jEImult = 1
    jIImult = 1

usejEE = p['jEE'] / p['nIncExc'] * vTauExcOverMS * jEEmult
usejIE = p['jIE'] / p['nIncExc'] * vTauInhOverMS * jIEmult
usejEI = p['jEI'] / p['nIncInh'] * vTauExcOverMS * jEImult
usejII = p['jII'] / p['nIncInh'] * vTauInhOverMS * jIImult

synapsesEE.jEE = usejEE
synapsesIE.jIE = usejIE
synapsesEI.jEI = usejEI
synapsesII.jII = usejII

In [9]:
print(usejEE, usejIE, usejEI, usejII, )

143.49509118 pA 123.92226015 pA 136.50692497 pA 101.08023944 pA


In [10]:
N.add(synapsesEE, synapsesIE, synapsesEI, synapsesII)

# create monitors
spikeMonExc = SpikeMonitor(unitsExc[:p['nExcSpikemon']])
spikeMonInh = SpikeMonitor(unitsInh[:p['nInhSpikemon']])
stateMonExc = StateMonitor(unitsExc,
                           p['recordStateVariables'],
                           record=p['indsRecordStateExc'],
                           clock=defaultclock)
stateMonInh = StateMonitor(unitsInh,
                           p['recordStateVariables'],
                           record=p['indsRecordStateInh'],
                           clock=defaultclock)
N.add(spikeMonExc, spikeMonInh, stateMonExc, stateMonInh)

In [11]:
tauRiseE = p['tauRiseExc']
tauFallE = p['tauFallExc']
tauRiseI = p['tauRiseInh']
tauFallI = p['tauFallInh']
tauAdapt = p['adaptTau']
tauRiseEOverMS = p['tauRiseExc'] / ms
tauRiseIOverMS = p['tauRiseInh'] / ms
vTauExcOverMS = p['membraneCapacitanceExc'] / p['gLeakExc'] / ms
vTauInhOverMS = p['membraneCapacitanceInh'] / p['gLeakInh'] / ms

N.run(p['duration'], report=p['reportType'], report_period=p['reportPeriod'], profile=p['doProfile'])

Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 1s


In [12]:
useDType = np.single

timeArray = np.arange(0, float(p['duration']), float(p['dt']), dtype=useDType)

spikeMonExcT = np.array(spikeMonExc.t, dtype=useDType)
spikeMonExcI = np.array(spikeMonExc.i, dtype=useDType)
spikeMonInhT = np.array(spikeMonInh.t, dtype=useDType)
spikeMonInhI = np.array(spikeMonInh.i, dtype=useDType)
stateMonExcV = np.array(stateMonExc.v / mV, dtype=useDType)
stateMonInhV = np.array(stateMonInh.v / mV, dtype=useDType)

stateMonExcSE = np.array(stateMonExc.sE / pA, dtype=useDType)
stateMonInhSE = np.array(stateMonInh.sE / pA, dtype=useDType)
stateMonExcSI = np.array(stateMonExc.sI / pA, dtype=useDType)
stateMonInhSI = np.array(stateMonInh.sI / pA, dtype=useDType)

In [13]:
f, ax = plt.subplots()
ax.plot(timeArray, stateMonExcV[0, :])
ax.plot(timeArray, stateMonInhV[0, :])

In [14]:
f, ax = plt.subplots()
ax.plot(timeArray, stateMonExcSE[0, :])
ax.plot(timeArray, stateMonExcSI[0, :])

In [15]:
f, ax = plt.subplots()
ax.plot(timeArray, stateMonExcSE[0, :] - stateMonExcSI[0, :])

In [445]:
sESumExc = (stateMonExc.sE).sum() * p['dt'] * 1e3
sESumInh = (stateMonInh.sE).sum() * p['dt'] * 1e3
sISumExc = (stateMonExc.sI).sum() * p['dt'] * 1e3
sISumInh = (stateMonInh.sI).sum() * p['dt'] * 1e3

In [446]:
print(sESumExc, sESumInh, sISumExc, sISumInh, )

271.79170679 nC 234.71912744 nC 188.9700735 nC 139.92799472 nC


In [447]:
print(usejEE, usejIE, usejEI, usejII, )

143.49509118 pA 123.92226015 pA 136.50692497 pA 101.08023944 pA


In [448]:
# time integral of sE and sI
sESumExc = stateMonExcSE[0, :].sum() * p['dt'] / ms * ncoulomb
sESumInh = stateMonInhSE[0, :].sum() * p['dt'] / ms * ncoulomb
sISumExc = stateMonExcSI[0, :].sum() * p['dt'] / ms * ncoulomb
sISumInh = stateMonInhSI[0, :].sum() * p['dt'] / ms * ncoulomb

In [449]:
print(sESumExc, sESumInh, sISumExc, sISumInh, )  # in pC

271.7917 uC 234.719125 uC 188.970075 uC 139.928 uC


In [444]:
print(sESumExc / numSpikesExc, sESumInh / numSpikesExc, sISumExc / numSpikesInh, sISumInh / numSpikesInh, )

NameError: name 'numSpikesExc' is not defined

In [18]:
print(usejEE, usejIE, usejEI, usejII, )

143.49509118 pA 123.92226015 pA 136.50692497 pA 101.08023944 pA


In [21]:
usejEE

140. * pamp

In [24]:
usejEI

140. * pamp

In [18]:
sISumExc

19600.0

In [19]:
spikeMonExcT.size / p['duration']

4. * hertz

In [20]:
spikeMonInhT.size / p['duration']

16. * hertz

In [24]:
doEmpiricalTest = True

In [25]:
if doEmpiricalTest:
    pForEphys = p.copy()
    if 'iExtRange' not in pForEphys:
        pForEphys['recordStateVariables'] = ['v', ]
        pForEphys['propInh'] = 0.5
        pForEphys['duration'] = 1 * second
        pForEphys['iExtRange'] = np.linspace(0, .3, 3001) * nA
    JEN = JercogEphysNetwork(pForEphys)
    JEN.build_classic()
    JEN.run()
    RE = ResultsEphys()
    RE.init_from_network_object(JEN)
    RE.calculate_thresh_and_gain()
    
    p['threshExc'] = RE.threshExc
    p['threshInh'] = RE.threshInh
    p['gainExc'] = RE.gainExc
    p['gainInh'] = RE.gainInh

Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 1s


In [26]:
# average net positive current to sustain set FR

In [30]:
p['threshExc'] = RE.threshExc
p['threshInh'] = RE.threshInh
p['gainExc'] = RE.gainExc
p['gainInh'] = RE.gainInh

In [31]:
print(RE.threshExc, RE.threshInh, RE.gainExc, RE.gainInh, )

124.1 pA 135.1 pA 0.34659091 Hz/pA 1.50909091 Hz/pA


In [155]:
float(p['setUpFRExc'])

5.0

In [156]:
p['duration']

1. * second

In [243]:
usejEE

143.49509118 * pamp

In [253]:
# rough estimates
sumExcInputToExc = nExc * usejEE * float(p['setUpFRExc']) * p['duration']
sumExcInputToInh = nExc * usejIE * float(p['setUpFRExc']) * p['duration']

In [254]:
# rough estimates
sumInhInputToExc = nInh * usejEI * float(p['setUpFRInh']) * p['duration']
sumInhInputToInh = nInh * usejII * float(p['setUpFRInh']) * p['duration']

In [255]:
print(sumExcInputToExc, sumExcInputToInh, sumInhInputToExc, sumInhInputToInh)

286.99018236 nC 247.84452029 nC 191.10969495 nC 141.51233522 nC


In [256]:
print(sESumExc, sESumInh, sISumExc, sISumInh)

271.79170679 nC 234.71912744 nC 188.9700735 nC 139.92799472 nC


In [205]:
# !!! my calculation is somewhat close

In [206]:
sESumExc - sISumExc

82.82163329 * ncoulomb

In [207]:
sESumInh - sISumInh

94.79113272 * ncoulomb

In [340]:
# net charge needed to sustain the setpoint FR
sustainingChargeExc = 83 * ncoulomb
sustainingChargeInh = 95 * ncoulomb

In [341]:
# translating that back into a weight
proposedjEI = (sESumExc - sustainingChargeExc) / nInh / float(p['setUpFRInh']) / p['duration']
proposedjII = (sESumInh - sustainingChargeInh) / nInh / float(p['setUpFRInh']) / p['duration']

In [342]:
print(proposedjEI, proposedjII)

134.85121914 pA 99.79937675 pA


In [211]:
# doing the same as above, but for the weights

In [442]:
jEESum = nExc * usejEE * float(p['setUpFRExc'])
jIESum = nExc * usejIE * float(p['setUpFRExc'])

In [443]:
print(jEESum, jIESum)

286.99018236 nA 247.84452029 nA


In [345]:
jEISumgood = sISumExc / p['duration']
jIISumgood = sISumInh / p['duration']

In [346]:
print(jEISumgood, jIISumgood)

188.9700735 nA 139.92799472 nA


In [347]:
jEESum - jEISumgood

98.02010886 * namp

In [348]:
jIESum - jIISumgood

107.91652557 * namp

In [349]:
# is this dependent on any chosen parameters here?
# and how can this number be calculated analytically?

sustainingWeightImbalanceExc = 98 * nA
sustainingWeightImbalanceInh = 108 * nA

In [350]:
proposedjEISum = (jEESum - sustainingWeightImbalanceExc)
proposedjIISum = (jIESum - sustainingWeightImbalanceInh)
proposedjEI = proposedjEISum / nInh / float(p['setUpFRInh'])
proposedjII = proposedjIISum / nInh / float(p['setUpFRInh'])

In [351]:
jEISumgood / jEESum

0.6584548361290394

In [352]:
jIISumgood / jIESum

0.5645797395814799

In [353]:
# is this dependent on any chosen parameters here?
# and how can this number be calculated analytically?

sustainingWeightRatioExc = 0.658
sustainingWeightRatioInh = 0.564

In [354]:
proposedjEISum = (jEESum * sustainingWeightRatioExc)
proposedjIISum = (jIESum * sustainingWeightRatioInh)
proposedjEI = proposedjEISum / nInh / float(p['setUpFRInh'])
proposedjII = proposedjIISum / nInh / float(p['setUpFRInh'])

In [355]:
jEESum

286.99018236 * namp

In [356]:
jEESum * p['setUpFRExc'] / p['setUpFRInh'] - jEISumgood

-86.4735798 * namp

In [357]:
jIESum * p['setUpFRExc'] / p['setUpFRInh'] - jIISumgood

-51.41209462 * namp

In [436]:
# is this dependent on any chosen parameters here?
# and how can this number be calculated analytically?

slopeExc = p['setUpFRExc'] / p['setUpFRInh']
slopeInh = p['setUpFRExc'] / p['setUpFRInh']
offsetExc = 86.5 * nA
offsetInh = 51.4 * nA

In [437]:
proposedjEISum = (offsetExc + jEESum * slopeExc)
proposedjIISum = (offsetInh + jIESum * slopeInh)
proposedjEI = proposedjEISum / nInh / float(p['setUpFRInh'])
proposedjII = proposedjIISum / nInh / float(p['setUpFRInh'])

In [438]:
print(proposedjEI, proposedjII)

134.9974955 pA 99.93992865 pA


In [439]:
print(usejEI, usejII)

136.50692497 pA 101.08023944 pA


In [150]:
# how to get it back to the weight?

In [258]:
sISumExc / nInh / float(p['setUpFRInh'])

134.97862393 * pcoulomb

In [259]:
print(sISumExc, sISumInh)

188.9700735 nC 139.92799472 nC


In [ ]:
# thi

In [284]:
p['setUpFRExc'] / p['setUpFRInh']

0.35714285714285715

In [377]:
p['setUpFRExc'] / p['setUpFRInh'] * jEESum

102.4964937 * namp

In [386]:
p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] * p['nExc'] * Hz

0.82435597 * namp

In [384]:
(p['threshExc'] * p['nExc']) / p['setUpFRInh'] * Hz

7.09142857 * namp

In [268]:
(p['gainExc'] / p['nExc'])

0.00043324 * hertz / pamp

In [326]:
jEESum

286.99018236 * namp

In [394]:
p['setUpFRExc'] / p['setUpFRInh'] * jEESum

102.4964937 * namp

In [397]:
p['setUpFRExc'] / p['setUpFRInh'] / (p['gainExc'] / p['nExc']) * Hz

0.82435597 * namp

In [391]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * jEESum - \
                   p['setUpFRExc'] / p['setUpFRInh'] / (p['gainExc'] / p['nExc']) * Hz - \
                   (p['threshExc'] * p['nExc']) / p['setUpFRInh'] * Hz # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * jIESum - \
                   1 / (p['gainInh'] / p['nInh']) * Hz - \
                   (p['threshInh'] * p['nInh']) / p['setUpFRInh'] * Hz

In [392]:
sumInhInputToExc1

95.40506384 * namp

In [393]:
sumInhInputToInh1

86.58589679 * namp

In [322]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * jEESum - \
                   p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc']  - \
                   p['threshExc'] / p['setUpFRInh']  # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * jIESum - \
                   1 / p['gainInh'] - \
                   p['threshInh'] / p['setUpFRInh']

DimensionMismatchError: Cannot calculate 102.4964937 nA - 1.03044496 pC, units do not match (units are A and C).

In [323]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * jEESum * second - \
                   p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] - \
                   p['threshExc'] / p['setUpFRInh']  # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * jIESum * second - \
                   1 / p['gainInh'] - \
                   p['threshInh'] / p['setUpFRInh']

In [454]:
p['setUpFRExc'] / p['setUpFRInh'] * jEESum / Hz

102.4964937 * ncoulomb

In [453]:
p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc']

1.03044496 * pcoulomb

In [455]:
p['threshExc'] / p['setUpFRInh']

8.86428571 * pcoulomb

In [456]:
p['gainExc']

0.34659091 * hertz / pamp

In [457]:
p['gainInh']

1.50909091 * hertz / pamp

In [458]:
p['threshExc']

124.1 * pamp

In [459]:
p['threshInh']

135.1 * pamp

In [ ]:
WEI = m * WEE + b

In [462]:
m = p['setUpFRExc'] / p['setUpFRInh']
m

0.35714285714285715

In [450]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * jEESum / Hz - \
                   p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] - \
                   p['threshExc'] / p['setUpFRInh']  # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * jIESum / Hz - \
                   1 / p['gainInh'] - \
                   p['threshInh'] / p['setUpFRInh']

In [325]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * usejEE * float(p['setUpFRExc']) - \
                   p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] - \
                   p['threshExc'] / p['setUpFRInh']  # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * (jIESum ) - \
                   1 / p['gainInh'] - \
                   p['threshInh'] / p['setUpFRInh']

In [336]:
p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc']

1.03044496 * pcoulomb

In [335]:
usejEE * float(p['setUpFRExc'])

0.71747546 * namp

In [329]:
jEESum * p['setUpFRExc']

1.43495091e-06 * second ** -1 * amp

In [328]:
p['setUpFRExc']

5. * hertz

In [325]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = 

In [425]:
print(p['threshExc'], p['gainExc'], p['threshInh'], p['gainInh'], )

124.1 pA 0.34659091 Hz/pA 135.1 pA 1.50909091 Hz/pA


In [426]:
# is this dependent on any chosen parameters here?
# and how can this number be calculated analytically?

slopeExc = p['setUpFRExc'] / p['setUpFRInh']
slopeInh = p['setUpFRExc'] / p['setUpFRInh']
offsetExc = 86.5 * nA
offsetInh = 51.4 * nA

In [440]:
offsetExc / float(p['setUpFRInh'])

6.17857143 * namp

In [441]:
offsetInh / float(p['setUpFRInh'])

3.67142857 * namp

In [427]:
proposedjEISum = (offsetExc + jEESum * slopeExc)
proposedjIISum = (offsetInh + jIESum * slopeInh)
proposedjEI = proposedjEISum / nInh / float(p['setUpFRInh'])
proposedjII = proposedjIISum / nInh / float(p['setUpFRInh'])

In [314]:
p['threshExc']

124.1 * pamp

In [316]:
offsetExc / p['nInh'] / float(p['setUpFRInh'])

30.89285714 * pamp

In [311]:
p['threshExc'] / (offsetExc / p['nInh'] / float(p['setUpFRInh']))

4.017109826589595

In [317]:
offsetInh / p['nInh'] / float(p['setUpFRInh'])

18.35714286 * pamp

In [313]:
p['threshInh'] / (offsetInh / p['nInh'] / float(p['setUpFRInh']))

7.35953307392996

In [289]:
p['setUpFRExc'] / p['setUpFRInh'] * jEESum

102.4964937 * namp

In [292]:
p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] + p['threshExc'] / p['setUpFRInh']

9.89473068 * pcoulomb

In [293]:
(1 / p['gainInh'] + p['threshInh'] / p['setUpFRInh'])

10.3126506 * pcoulomb

In [237]:
# balance calculation based on Wilson-Cowan, etc
sumInhInputToExc1 = p['setUpFRExc'] / p['setUpFRInh'] * sumExcInputToExc - \
                   p['setUpFRExc'] / p['setUpFRInh'] / p['gainExc'] - \
                   p['threshExc'] / p['setUpFRInh']  # amp
sumInhInputToInh1 = p['setUpFRExc'] / p['setUpFRInh'] * sumExcInputToInh - \
                   1 / p['gainInh'] - \
                   p['threshInh'] / p['setUpFRInh']

In [238]:
sumInhInputToExc1

102.48659897 * ncoulomb

In [239]:
sumInhInputToInh1

88.50558745 * ncoulomb

In [281]:
# is this dependent on any chosen parameters here?
# and how can this number be calculated analytically?

slopeExc = p['setUpFRExc'] / p['setUpFRInh']
slopeInh = p['setUpFRExc'] / p['setUpFRInh']
offsetExc = 86.5 * nA
offsetInh = 51.4 * nA

In [399]:
offsetExc / nExc

216.25 * pamp

In [404]:
offsetExc / nInh

0.865 * namp

In [411]:
(offsetExc / nExc) / p['threshExc']

1.742546333601934

In [407]:
(offsetExc / nInh) / p['threshExc']

6.970185334407736

In [400]:
p['threshExc']

124.1 * pamp

In [412]:
(offsetInh / nExc) / p['threshInh']

0.9511472982975575

In [421]:
p['threshInh'] * nInh

13.51 * namp

In [422]:
offsetExc / (p['threshExc'] * nExc)

1.7425463336019342

In [423]:
offsetInh / (p['threshInh'] * nInh)

3.8045891931902296

In [409]:
(offsetInh / nInh) / p['threshInh']

3.80458919319023

In [403]:
p['threshInh']

135.1 * pamp

In [43]:
netSustainingCurrentExc = RE.p['iExtRange'][np.where(RE.spikeMonExcC == p['setUpFRExc'] * second)[0]].mean()
netSustainingCurrentInh = RE.p['iExtRange'][np.argmin(np.abs(RE.spikeMonInhC - p['setUpFRInh'] * second))]

print(netSustainingCurrentExc, netSustainingCurrentInh)

132.25 pA 135.1 pA


In [37]:
netSustainingCurrentExc

132.25 * pamp

In [38]:
sumExcInputToExc

286.99018236 * namp

In [128]:
# balance calculation based on sustaining current idea (new)
sumInhInputToExc2 = sumExcInputToExc - netSustainingCurrentExc
sumInhInputToInh2 = sumExcInputToInh - netSustainingCurrentInh

In [129]:
indivWeightEI2 = sumInhInputToExc2 / nInh / (p['setUpFRInh'] * second)
indivWeightII2 = sumInhInputToInh2 / nInh / (p['setUpFRInh'] * second)

In [130]:
print(indivWeightEI2, indivWeightII2)

199.90553571 pA 178.47492857 pA


In [131]:
print(sumExcInputToExc, sumInhInputToExc1, sumExcInputToInh, sumInhInputToInh1,)

280. nA 99.99010527 nA 250. nA 89.27540164 nA


In [132]:
print(sumExcInputToExc, sumInhInputToExc2, sumExcInputToInh, sumInhInputToInh2,)

280. nA 279.86775 nA 250. nA 249.8649 nA


In [113]:
roughInhInputToExc = (nInh * usejEI * p['setUpFRInh'] * second)
roughInhInputToInh = (nInh * usejII * p['setUpFRInh'] * second)

In [114]:
sumInhInputToExc / roughInhInputToExc

0.5101532703245233

In [115]:
sumInhInputToInh / roughInhInputToInh

0.6376809810671256